In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Pending

In [ ]:
# from scipy.io import loadmat

# link = r"D:\external_data\Experiment4"
# filename = r"\Dataset_PWR_WiFi.mat"
# directory = link + filename

# mat = loadmat(directory)

In [ ]:
# # import experimental dataset 2
# folderpath2 = "D:/external_data/Experiment3/csv_files/exp_2"  # CHANGE THIS IF THE PATH CHANGED
# df_exp2 = import_clean_data('exp2',folderpath2)

# lab

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from data.process_data import stacking
import matplotlib.pyplot as plt



def visual_spectrogram(img,title='spectrogram'):
    
    if len(img.shape) == 3:
        
        img.squeeze()
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(img)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
    plt.figure(figsize = (15,50))
    plt.imshow(img,cmap='jet')
    plt.title(title)
    plt.show()
    
    return 

In [ ]:
import torchvision.models as models
        
#visualize weights for alexnet - first conv layer
alexnet = models.alexnet(pretrained=False)

# Store all conv layer in alexnet
k = []
for m in alexnet.modules():
    if isinstance(m, nn.Conv2d):
        print(m.weight.data.shape)
        k.append(m)
    else:  
        continue


In [ ]:
def visual_single_filter(weight):
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(weight)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
#     plt.figure(figsize = (15,10))
    plt.imshow(weight,cmap='jet')
#     plt.show()
    
    return 

def visualise_filters(layer,figsize=(20,15)):
    
    weights = layer.weight.data

    if weights.shape[0] > weights.shape[1]:
        
        weights = weights.transpose(1,0)
        
    rows = weights.shape[0]
    
    cols = weights.shape[1]
    
    axes = []
    
    fig= plt.figure()

    for n in range(rows):

        for c in range(cols):
            
            axes.append(fig.add_subplot(rows, cols, n*cols+c+1))

            single_filter = weights[n,c]
        
            visual_single_filter(single_filter)
            
#     fig.tight_layout()    
    
    plt.show()     
    
    return 

visualise_filters(example)

# model factory

In [1]:
import time
from time import gmtime, strftime

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

##### extra setting

In [3]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

torch.cuda.device_count()

torch.cuda.set_device(device)

cuda:0


In [5]:
np.random.seed(1024)
torch.manual_seed(1024)
# torch.set_deterministic(True)

### 0. Default

In [7]:
# model 
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        pass
    
    def forward(self,X):
        return X

### 1. Section

In [6]:
from models import ED_module, Lambda, Classifier
from torchsummary import summary

In [7]:
class Encoder(nn.Module):
    """
    Encoder for spectrogram (1,65,65), 3 layer
    """
    def __init__(self,num_filters):
        super(Encoder, self).__init__()
        l1,l2,l3 = num_filters
        ### 1st ###
        self.conv1 = nn.Conv2d(1,l1,kernel_size=5,stride=2)
        self.norm1 = nn.BatchNorm2d(l1)
        self.actv1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))
        ### 2nd ###
        self.conv2 = nn.Conv2d(l1,l2,kernel_size=3,stride=2)
        self.norm2 = nn.BatchNorm2d(l2)
        self.actv2 = nn.ReLU()
        self.pool2 = Lambda(lambda x:x) # nn.MaxPool2d(kernel_size=(2,2))
        ### 3rd ###
        self.conv3 = nn.Conv2d(l2,l3,kernel_size=2,stride=2)
        self.norm3 = Lambda(lambda x:x)
        self.actv3 = nn.Tanh()
        self.pool3 = nn.AvgPool2d(kernel_size=(2,2))
        
    def forward(self,X):
        X = self.pool1(self.actv1(self.norm1(self.conv1(X))))
        X = self.pool2(self.actv2(self.norm2(self.conv2(X))))
        X = self.pool3(self.actv3(self.norm3(self.conv3(X))))
        X = torch.flatten(X, 1)
        # print(X.shape)
        return X
    
# mdl = Encoder([64,128,512]).cuda()
# summary(mdl,(1,65,65),batch_size=128)

In [8]:
class Projection_head(nn.Module):
    """
    Projection head: 
    
    head: linear/mlp
    """
    def __init__(self,dim_in,feat_dim,head='linear'):
        super(Projection_head, self).__init__()
        if head == 'linear':
            self.head = nn.Linear(dim_in, feat_dim)
        elif head == 'mlp':
            self.head = nn.Sequential(
                nn.Linear(dim_in, dim_in),
                nn.ReLU(inplace=True),
                nn.Linear(dim_in, feat_dim)
            )

    def forward(self,X):
        X = self.head(X)
        X = F.normalize(X, dim=1)
        X = X.unsqueeze(1)
        return X

### 2. calling function

In [60]:
del mdl
torch.cuda.empty_cache()

NameError: name 'mdl' is not defined

In [17]:
assert False

AssertionError: 

# Test

In [18]:
from data import process_data
from data.import_data import import_clean_data
from data.datasetobj import DatasetObject
# from models.train import * 

### 0. shortcut

### 1. Data

In [9]:
from data.spectrogram import import_data
from data.process_data import label_encode, create_dataloaders
from sklearn.model_selection import train_test_split

In [10]:
columns = [f"col_{i+1}" for i in range(501)]
window_size=65
slide_size=30
dirc = "E:/external_data/Experiment2/spectrogram_data_by_activity_csv"

X,y  = import_data(dirc,columns=columns,window_size=window_size,slide_size=slide_size)
X.shape

kneel
lie
pickup
sit
stand
standff
walk


(5730, 65, 65)

In [12]:
X = X.reshape(*X.shape,1).transpose(0,3,1,2)

In [13]:
y,lb = label_encode(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test, train_batch_sizes=128, test_batch_sizes=200)

In [11]:
assert False

AssertionError: 

### 2. Model

In [ ]:
# from models.baseline import Lambda, Classifier,CNN_module

In [15]:
from models.self_supervised import DataAugmentation

data_aug = DataAugmentation((65,65))

In [44]:
def create_model():
    enc = Encoder([64,128,512])
    clf = Projection_head(512,128,head='linear')
    model = ED_module(encoder=enc,decoder=clf)
    return model

model = create_model()

In [45]:
from losses import SupConLoss

criterion = SupConLoss(temperature=0.1,base_temperature=0.1)
optimizer = torch.optim.SGD(list(model.parameters()), lr=0.0005)

In [43]:
del model ,criterion , optimizer
torch.cuda.empty_cache()

In [30]:
print(torch.cuda.memory_summary(device=device, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  521317 KB |     994 MB |  124630 MB |  124120 MB |
|       from large pool |  516097 KB |     989 MB |  122986 MB |  122482 MB |
|       from small pool |    5220 KB |       7 MB |    1643 MB |    1638 MB |
|---------------------------------------------------------------------------|
| Active memory         |  521317 KB |     994 MB |  124630 MB |  124120 MB |
|       from large pool |  516097 KB |     989 MB |  122986 MB |

### 3. Training

In [43]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# from train import train


In [31]:
start = 1
end = 100
parallel = True

In [ ]:
def pretrain(model,train_loader,start,end,optimizer,criterion,data_aug,parallel=True):

    # Check device setting
    if parallel == True:
        print('GPU')
        model = model.to(device)
        data_aug = data_aug.to(device)

    else:
        print('CPU')
        model = model.cpu()

    print('Start Training')
    record = {'train':[],'validation':[]}
    i = start

    #Loop
    while i <= end:

        print(f"Epoch {i}: ", end='')

        for b, (X, y) in enumerate(train_loader):

            if parallel == True:

                X = X.to(device)
                y = y.to(device)

            print(f">", end='')

            optimizer.zero_grad()
            
            batch_size = X.shape[0]

            X = torch.cat(data_aug(X), dim=0)

            logits = model(X)

            logits = torch.split(logits, [batch_size, batch_size], dim=0)

            logits = torch.cat((logits[0].unsqueeze(1),logits[1].unsqueeze(1)),dim=1)

            loss = criterion(logits,y)

            loss.backward()

            optimizer.step()

        # One epoch completed
        l = loss.tolist()
        record['train'].append(l)
        print(f' loss: {l} ',end='')

    # if (test_loader != None) and i%100 ==0 :
    #     acc = short_evaluation(model,test_loader,parallel)
    #     record['validation'].append(acc)
    #     print(f' accuracy: {acc}')
    # else:
        print('')

        i += 1
        
        del X,y,logits

    model = model.cpu()
    
    return model

model = pretrain(model,train_loader,start,end,optimizer,criterion,data_aug,parallel)

In [60]:
def freeze_network(model):
    for _, p in model.named_parameters():
        p.requires_grad = False
    return model

def create_model(enc):
    if enc == None:
        enc = Encoder([64,128,512])
    else:
        enc = freeze_network(enc)
    clf = nn.Linear(512,7)
    model = ED_module(encoder=enc,decoder=clf)
    return model

cnn = create_model(enc)

In [62]:
from train import train

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(list(cnn.parameters()), lr=0.0005)

start = 1
end = 1
parallel = True


cnn = train(cnn,train_loader,criterion,optimizer,end,start,test_loader,parallel)



GPU
Start Training
Epoch 1: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loss: 1.8947129249572754 


In [ ]:
assert False

### 4. Testing

In [ ]:
cmtx,cls = evaluation(model,test_loader)
cmtx = cmtx_table(cmtx,label_encoder=None)
cmtx

In [ ]:
train_loss = pd.DataFrame(record['train'],columns=['train_loss'])

### 5. Saving

In [ ]:
from models.train import save_checkpoint, make_directory

In [ ]:
path = make_directory('V2cSimCLR',epoch=500,filepath='./record/')
train_loss.to_csv(path+'_loss.csv')
# cls.to_csv(path+'_report.csv')
# cmtx.to_csv(path+'_cmtx.csv')

In [ ]:
path = make_directory('V2cSimCLR_pretrain',epoch=500)
save_checkpoint(model, optimizer, 500, path)